First, let's get a list of all the photos in our dataset

In [14]:
import glob
img_list = glob.glob("/floyd/input/cars/*.jpg")

Now let's loop through the actual images and create an array of values

In [15]:
import os
from PIL import Image
from numpy import asarray
import numpy as np
batch_len = 20
data_array = []
dataset_array = []
for i in range(7500):
    im = Image.open(img_list[i])
    resized_image = im.resize((288,288))
    image_array = asarray(resized_image)
    if image_array.shape == (288,288,3):
        data_array.append(image_array)
    if len(data_array) == batch_len:
        #print(i)
        data_array = asarray(data_array)
        dataset_array.append(data_array)
        data_array = []


In [16]:
new_data_set_array = []

for X_batch in dataset_array:
    new_data_set_array.append(X_batch.astype(np.float32) / 255.0)
    
dataset_array = []


Now we can start work on the Nueral Network

In [1]:
import tensorflow as tf
from tensorflow import keras

In [11]:
num_features = 100
generator = keras.models.Sequential([
    keras.layers.Dense(10*10, input_shape=[num_features]),
    keras.layers.Dense(100*100),
    keras.layers.Dense(12*12*120),
    keras.layers.Reshape([12, 12, 120]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(128, (5,5), (4,4), padding="same", activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(64, (5,5), (3,3), padding="same", activation="selu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(3, (5,5), (2,2), padding="same", activation="tanh"),
])

In [12]:
noise = tf.random.normal(shape=[1, num_features])
generated_images = generator(noise, training=False)
generated_images.shape


TensorShape([1, 288, 288, 3])

Discriminator Network

In [16]:
discriminator = keras.models.Sequential([
    keras.layers.Conv2D(256, (5,5), (2,2), padding="same", input_shape=[288, 288, 3]),
    keras.layers.LeakyReLU(0.2),
    keras.layers.Dropout(0.3),
    keras.layers.Conv2D(128, (5,5), (2,2), padding="same"),
    keras.layers.LeakyReLU(0.2),
    keras.layers.Conv2D(128, (5,5), (2,2), padding="same"),
    keras.layers.LeakyReLU(0.2),
    keras.layers.Dropout(0.3),
    keras.layers.Flatten(),
    keras.layers.Dense(50*100),
    keras.layers.Dense(25*25),
    keras.layers.Dense(1, activation='sigmoid')
])

ResourceExhaustedError: OOM when allocating tensor with shape[165888,10000] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:RandomUniform]

In [21]:
decision = discriminator(generated_images)
print(decision)

tf.Tensor([[0.49987498]], shape=(1, 1), dtype=float32)


In [22]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan = keras.models.Sequential([generator, discriminator])
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [23]:
from IPython import display
from tqdm import tqdm
batch_size = 1
seed = tf.random.normal(shape=[batch_size, 100])

In [24]:
from tqdm import tqdm
from IPython.display import display
from PIL import Image
saved_images = []
def train_dcgan(gan, dataset, batch_size, num_features, epochs=5):
    generator, discriminator = gan.layers
    for epoch in tqdm(range(epochs)):
        print("Epoch {}/{}".format(epoch + 1, epochs))
        for X_batch in (dataset):
            #print("Started Batch...")
            noise = tf.random.normal(shape=[batch_size, num_features])
            generated_images = generator(noise)           
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            noise = tf.random.normal(shape=[batch_size, num_features])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
            # Produce images for the GIF as we go
            #display.clear_output(wait=True)
        #display.clear_output(wait=True)
        saved_images.append(generator(seed))
        upgraded_noise = generator(seed).numpy()
        im_data = (upgraded_noise.squeeze()*255).astype(np.uint8)
        im = Image.fromarray(im_data, mode='RGB')
        display(im) 
    #display.clear_output(wait=True)
    saved_images.append(generator(seed))

In [25]:
# for i in new_data_set_array:
#     print(i.shape)

In [1]:
train_dcgan(gan, new_data_set_array, batch_len, 100, 100)

NameError: name 'train_dcgan' is not defined